In [ ]:
import os
import sys
import random
import pickle
import numpy as np
import matplotlib.pyplot as plt
import torch
from numpy import linalg as la
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import directed_hausdorff

sys.path.insert(0, '../../../methods')
from mode_omp import *

In [ ]:
# random seed
seed = 1234
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
snr_list = [-20,-15,-10,-5,0,5,10,15,20,25,30]
T_list = [100,200,500,1000,2000,5000,10000]
rmse_list1 = []
rmse_list2 = []
hausdorff_list1 = []
hausdorff_list2 = []
pdet_list1 = []
pdet_list2 = []
sn_acc_list1 = []
sn_acc_list2 = []

for snr in snr_list:
    # load data
    data = np.load(f'../../../data/experiment_3/scenario_1/data_test_snr{snr}_t1000.npy', allow_pickle=True)

    # get labels and data seperately
    truths = [s['label'] for s in data]
    sensor_pos = [s['sensor_pos'] for s in data]
    data = [s['signals'] for s in data]

    # apply mode-omp algorithm
    res = 1
    p_list = []
    r_list = []
    h_list = []
    n_list = []

    for i in range(len(data)):
        X = data[i]
        M = X.shape[0]
        N = truths[i].shape[0]
        pos = sensor_pos[i]
        truths[i] = truths[i].reshape(-1,)

        preds = mode_omp(X, pos, M, snr)
        preds = np.sort(preds)
        truths[i] = np.sort(truths[i])

        hausdorff1 = directed_hausdorff(preds.reshape(-1,1), truths[i].reshape(-1,1))[0]
        hausdorff2 = directed_hausdorff(truths[i].reshape(-1,1), preds.reshape(-1,1))[0]
        h_list.append(max(hausdorff1, hausdorff2))

        n_list.append(len(truths[i])==len(preds))

    hausdorff = np.mean(h_list)
    sn_acc = np.sum(n_list) / len(n_list)
    hausdorff_list1.append(hausdorff)
    sn_acc_list1.append(sn_acc)

    print(f"snr {snr}dB, T {1000}, test-hausdorff {hausdorff:.4f}, test-sn-acc {sn_acc:.4f}")

    with open(f'../../../results/experiment_3/scenario_1/mode_omp_hamid2023_preds_snr{snr}_t1000', "wb") as fp:
        pickle.dump(p_list, fp)
    with open(f'../../../results/experiment_3/scenario_1/mode_omp_hamid2023_truths_snr{snr}_t1000', "wb") as fp:
        pickle.dump(truths, fp)


for T in T_list:
    # load data
    data = np.load(f'../../../data/experiment_3/scenario_1/data_test_snr-10_t{T}.npy', allow_pickle=True)

    # get labels and data seperately
    truths = [s['label'] for s in data]
    sensor_pos = [s['sensor_pos'] for s in data]
    data = [s['signals'] for s in data]

    # apply mode-omp algorithm
    res = 1
    p_list = []
    r_list = []
    h_list = []
    n_list = []

    for i in range(len(data)):
        X = data[i]
        M = X.shape[0]
        N = truths[i].shape[0]
        pos = sensor_pos[i]
        truths[i] = truths[i].reshape(-1,)

        preds = mode_omp(X, pos, M, -10)
        preds = np.sort(preds)
        truths[i] = np.sort(truths[i])

        hausdorff1 = directed_hausdorff(preds.reshape(-1,1), truths[i].reshape(-1,1))[0]
        hausdorff2 = directed_hausdorff(truths[i].reshape(-1,1), preds.reshape(-1,1))[0]
        h_list.append(max(hausdorff1, hausdorff2))

        n_list.append(len(truths[i])==len(preds))

    hausdorff = np.mean(h_list)
    sn_acc = np.sum(n_list) / len(n_list)
    hausdorff_list2.append(hausdorff)
    sn_acc_list2.append(sn_acc)

    print(f"snr -10dB, T {T}, test-hausdorff {hausdorff:.4f}, test-sn-acc {sn_acc:.4f}")

    with open(f'../../../results/experiment_3/scenario_1/mode_omp_hamid2023_preds_snr-10_t{T}', "wb") as fp:
        pickle.dump(p_list, fp)
    with open(f'../../../results/experiment_3/scenario_1/mode_omp_hamid2023_truths_snr-10_t{T}', "wb") as fp:
        pickle.dump(truths, fp)

np.save('../../../results/experiment_3/scenario_1/mode_omp_hamid2023_rmse1.npy', rmse_list1)
np.save('../../../results/experiment_3/scenario_1/mode_omp_hamid2023_rmse2.npy', rmse_list2)
np.save('../../../results/experiment_3/scenario_1/mode_omp_hamid2023_hausdorff1.npy', hausdorff_list1)
np.save('../../../results/experiment_3/scenario_1/mode_omp_hamid2023_hausdorff2.npy', hausdorff_list2)
np.save('../../../results/experiment_3/scenario_1/mode_omp_hamid2023_snacc1.npy', sn_acc_list1)
np.save('../../../results/experiment_3/scenario_1/mode_omp_hamid2023_snacc2.npy', sn_acc_list2)

In [ ]:
# plot rmse values
plt.figure()
plt.plot(snr_list, hausdorff_list1, '-o')
plt.title("rmse values for different snr levels")
plt.xlabel("snr (dB)")
plt.ylabel("rmse (deg)")
plt.legend(['mode-omp (hamid 2023)'])
plt.yscale("log")
plt.grid()

plt.figure()
plt.plot(T_list, hausdorff_list2, '-o')
plt.title("rmse values for different snapshot numbers")
plt.xlabel("T")
plt.ylabel("rmse (deg)")
plt.legend(['mode-omp (hamid 2023)'])
plt.yscale("log")
plt.grid()